# Gibbs Sampler

## Setup

In [1]:
import torch
import os
import numpy as np
from decimal import Decimal, getcontext
# os.chdir('/content/Economy-and-Population-Growth')

In [2]:
# !git pull origin

In [3]:
torch.cuda.is_available()

False

In [4]:
from Gibbs.GibbsSampler import GibbsSampler
from DataGeneration import generate_multiple_series
from Gibbs import Initialize
from Trends import Regressors
from Trends import LowFrequencyTrends
from Gibbs.Parameters import *
from Gibbs import Priors
import Gibbs.StepsHelper as StepsHelper
import time
import math

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## Iniatlizing parameters

In [6]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs.csv")
w = np.random.random(120)
w = w/np.sum(w)
w = torch.tensor(w, device=device)

In [7]:
n = 120
m = 25
l = 10
T = 150
q = 16
q_hat = q+15
R_hat = Regressors.find_regressors(T, q_hat).T

In [8]:
def initialize(data, w, n, m, l, T, q, q_hat, device):
  R_hat = Regressors.find_regressors(T, q_hat).T
  X_i = Initialize.initialize_X(data, q)
  F = Initialize.initialize_F(X_i, w, q_hat)
  S_m, sigma_m, Sigma_m = Initialize.initialize_S_m(T, R_hat, device)
  K = torch.tensor(Priors.group_factors(m, l), device=device)
  J = torch.tensor(Priors.group_factors(n, m), device=device)
  lambdas = Lambda_Parameters(n, m, device)
  kappas = Kappa_Parameters(n, m, l, device)
  p_parameters = P_Parameters(device)

  s_Da = Initialize.inverse_squared(0.03**2)
  A, Sigma_A = Initialize.init_Sigma_A(R_hat, T, q_hat, s_Da)

  f_0 = Priors.flat_prior(0, 10**6)
  mu_m = Priors.flat_prior(0, 10**6)
  mu_c = Priors.flat_prior(0, 10**6)
  omega_squared = Initialize.inverse_squared(1)

  U = U_Parameters(omega_squared, lambdas, kappas, R_hat, n, m, l, T, device)

  G = []
  for j in range(m):
      G_j = lambdas.lambda_g_j[j]*U.H[K[j]]+U.U_g[j]
      G.append(G_j)
  G = torch.stack(G)

  C = []
  i_1 = torch.zeros(q_hat+1, device=device)
  i_1[0] = 1
  for i in range(n):
      C_i = mu_c*i_1+lambdas.lambda_c_i[i]*G[J[i]]+U.U_c[i]
      C.append(C_i)
  C = torch.stack(C)
  Ys = []
  for i in range(n):
      Y_i, _ = LowFrequencyTrends.find_trends(data[i], q)
      Ys.append(Y_i)

  Y = np.array(Ys).flatten()
  Y = torch.tensor(Y, device=device)

  return X_i, F, S_m, sigma_m, Sigma_m, K, J, lambdas, kappas, p_parameters, s_Da, A, Sigma_A, f_0, mu_m, mu_c, omega_squared, U, G, C, Y

In [9]:
X_i, F, S_m, sigma_m, Sigma_m, K, J, lambdas, kappas, p_parameters, s_Da, A, Sigma_A, f_0, mu_m, mu_c, omega_squared, U, G, C, Y = initialize(data, w, n, m, l, T, q, q_hat, device)

## Step 1

Stable parameters:

In [10]:
dim = (q_hat+1)*n
Sigma = torch.zeros((dim, dim), device=device)
for i in range(n):
    Sigma[i*(q_hat+1):(i+1)*(q_hat+1), i*(q_hat+1):(i+1)*(q_hat+1)] = U.S_U_c[i]

In [11]:
I = torch.eye(q_hat+1, device=device)
ws = torch.kron(w.t(), I).t()
Delta = torch.eye(q_hat+1, device=device)*0.01**2

In [12]:
def step1(X_i, ws, Delta, Y, C, F, lambdas, n, device):
  Y0 = torch.matmul(w, C)
  i_1 = torch.zeros(q_hat+1, device=device)
  i_1[0] = 1
  mu_C = torch.cat([mu_c*i_1.t()+lambdas.lambda_c_i[i]*G[J[i]].t() for i in range(n)]).t()
  X = torch.flatten(X_i)
  B = StepsHelper.find_B(Y, X, device)
  Cs = torch.flatten(C)
  helper = torch.linalg.multi_dot([Sigma, B, torch.inverse(torch.linalg.multi_dot([B.t(), Sigma, B])), B.t()])
  V = Sigma-torch.matmul(helper, Sigma)
  normal_V = V/torch.norm(V)
  Z0 = StepsHelper.draw_independent_samples(mu_C.float(), normal_V.float(), n)
  Z1 = Z0-torch.matmul(helper, Z0-Cs.float())
  e_dist = torch.distributions.MultivariateNormal(Y0.float(), Delta.float())
  epsilon = e_dist.sample()
  ep = torch.repeat_interleave(epsilon, repeats=n)
  inv_help = torch.inverse(torch.linalg.multi_dot([ws.t().float(), V, ws.float()])+Delta)
  Cs = Z1-torch.linalg.multi_dot([V, ws.float(), inv_help, ws.t().float(), Z1-ep])
  CC = torch.reshape(Cs, (n, q_hat+1))
  XX = CC-F
  return XX, CC, Sigma, mu_C, Y0, X

In [13]:
X_i, C, Sigma, mu_C, Y0, X = step1(X_i, ws, Delta, Y, C, F, lambdas, n, device)

## Step 2

Standard parameters:

In [14]:
e = torch.kron(torch.ones(n, device=device).t(), I).t()

In [15]:
def step2(e, f_0, mu_m, sigma_m, Sigma_m, s_Da, Sigma_A, Sigma, Delta, mu_C, ws, X, Y0):
  i_1 = torch.zeros(q_hat+1, device=device)
  i_1[0] = 1
  i_2 = torch.zeros(q_hat+1, device=device)
  i_2[1] = 1
  mu_F = i_1*f_0+i_2*mu_m
  Sigma_F = sigma_m**3*Sigma_m+s_Da**2*Sigma_A
  V_F = torch.inverse(torch.inverse(Sigma_F)+torch.linalg.multi_dot([e.t().float(), torch.inverse(Sigma).float(), e.float()])+torch.inverse(Delta))
  helper = X.double()-mu_C.double()-torch.matmul(e.double(), mu_F.double()).double()
  add1 = torch.linalg.multi_dot([e.t().float(), torch.inverse(Sigma).float(), helper.float()])
  add2 = torch.matmul(torch.inverse(Delta).float(), torch.matmul(ws.t().float(), X.float()).float()-Y0.float()-mu_F.float())
  m_F = torch.matmul(V_F.float(), add1.float()+add2.float())
  dist = torch.distributions.MultivariateNormal(m_F.float(), StepsHelper.make_positive_definite(V_F).float())
  draw = dist.sample()
  FF = draw+mu_F
  Sigma_S = sigma_m**2*Sigma_m
  mult = FF-mu_F
  mean_S_m = torch.linalg.multi_dot([Sigma_S.float(), torch.inverse(Sigma_F).float(), mult.float()])
  var_S_m = Sigma_S-torch.linalg.multi_dot([Sigma_S.float(), torch.inverse(Sigma_F).float(), Sigma_S.float()])
  S_m_dist = torch.distributions.MultivariateNormal(mean_S_m.float(), StepsHelper.make_positive_definite(var_S_m.float()))
  SS_m = S_m_dist.sample()
  return FF, SS_m

In [16]:
F, S_m = step2(e, f_0, mu_m, sigma_m, Sigma_m, s_Da, Sigma_A, Sigma, Delta, mu_C, ws, X, Y0)

## Step 3

In [17]:
Sigma_g_j = []
inv_g_j = []
for i in range(m):
  g_j = omega_squared*kappas.kappa_g_j[i]**2*(1-lambdas.lambda_g_j[i]**2)*U.S_U_g[i]
  Sigma_g_j.append(g_j)
  inv_g_j.append(torch.inverse(g_j))

In [18]:
Sigma_c_i = []
inv_c_i = []
for i in range(n):
  c_i = omega_squared*kappas.kappa_c_i[i]**2*(1-lambdas.lambda_c_i[i]**2)*U.S_U_c[i]
  Sigma_c_i.append(c_i)
  inv_c_i.append(torch.inverse(c_i))

In [19]:
def iterate_step3(omega_squared, lambdas, inv_c_i, inv_sigma, J, ind, q_hat, C, U, device):
  find_ind = torch.where(J == ind)[0]
  sum1 = torch.zeros((q_hat+1, q_hat+1), device=device)
  for i in find_ind:
    sum1 += lambdas.lambda_c_i[i]**2*inv_c_i[i]
  V_G_ind = torch.inverse(inv_sigma+sum1)
  i_1 = torch.zeros(q_hat+1, device=device)
  i_1[0] = 1
  sum2 = torch.zeros(q_hat+1, device=device)
  for i in find_ind:
    sum2 += lambdas.lambda_c_i[i]*torch.matmul(inv_c_i[i].float(), C[i]-mu_c*i_1)

  m_G_ind = torch.matmul(V_G_ind, lambdas.lambda_g_j[ind]*torch.matmul(inv_sigma, U.H[K[ind]])+sum2)
  dist = torch.distributions.MultivariateNormal(m_G_ind, StepsHelper.make_positive_definite(V_G_ind))
  return dist.sample()

In [20]:
def step3(omega_squared, lambdas, inv_c_i, inv_g_j, J, q_hat, m, C, U, device):
  new_G = []
  for j in range(m):
    new_G.append(iterate_step3(omega_squared, lambdas, inv_c_i, inv_g_j[j], J, j, q_hat, C, U, device))

  return torch.stack(new_G)

In [21]:
G = step3(omega_squared, lambdas, inv_c_i, inv_g_j, J, q_hat, m, C, U, device)

## Step 4

In [22]:
Sigma_H_k = []
inv_h_k = []
for i in range(l):
  h_k = omega_squared*kappas.kappa_h_k[i]**2*U.S_h[i]
  Sigma_H_k.append(h_k)
  inv_h_k.append(torch.inverse(h_k))

In [23]:
def iterate_step4(lambdas, K, G, ind, inv_g_j, inv_sigma, q_hat, device):
  find_ind = torch.where(K == ind)[0]
  sum1 = torch.zeros((q_hat+1, q_hat+1), device=device)
  for i in find_ind:
    sum1 += lambdas.lambda_g_j[i]**2*inv_g_j[i]
  V_H_k = torch.inverse(inv_sigma+sum1)
  sum2 = torch.zeros(q_hat+1, device=device)
  for i in find_ind:
    sum2 += lambdas.lambda_g_j[i]*torch.matmul(inv_g_j[i], G[i])
  m_H_k = torch.matmul(V_H_k.float(), sum2.float())
  dist = torch.distributions.MultivariateNormal(m_H_k.float(), StepsHelper.make_positive_definite(V_H_k).float())
  return dist.sample()

In [24]:
def step4(lambdas, K, G, inv_g_j, inv_h_k, l, q_hat, device):
  new_H = []
  for k in range(l):
    new_H.append(iterate_step4(lambdas, K, G, k, inv_g_j, inv_h_k[k], q_hat, device))
  return new_H

In [25]:
U.H = step4(lambdas, K, G, inv_g_j, inv_h_k, l, q_hat, device)

## Step 5

In [26]:
def step5(q_hat, lambdas, C, G, J, inv_c_i):
  i_1 = torch.zeros(q_hat+1, device=device)
  i_1[0] = 1
  denominator = 0
  numerator = 0
  for i in range(n):
    add = C[i]-lambdas.lambda_c_i[i]*G[J[i]]
    numerator += torch.linalg.multi_dot([i_1.t(), inv_c_i[i].float(), add.float()]).item()
    denominator += torch.linalg.multi_dot([i_1.t().float(), inv_c_i[i].float(), i_1.float()]).item()

  mean_mu_c = numerator/denominator
  var_mu_c = 1/denominator
  return torch.normal(mean = mean_mu_c, std = torch.sqrt(torch.tensor(var_mu_c, device=device))).item()

In [27]:
mu_c = step5(q_hat, lambdas, C, G, J, inv_c_i)

## Step 6

In [28]:
def iterate_step6(omega_squared, p_parameters, kappa_param, lambda_param, S_U, mu_c, C_vec, q_hat, grid_points, device):
    # i_1 = torch.zeros(q_hat+1, device=device)
    # i_1[0] = 1
    # getcontext().prec = 50
    # eul = math.exp(1)
    # ps = []
    # exps = []
    # sum = Decimal('0')
    # for j in range(25):
    #     helper = C_vec-mu_c*i_1-grid_points[j]
    #     between = torch.inverse(omega_squared*kappa_param**2*(1-grid_points[j]**2)*S_U)
    #     exponent = -0.5*torch.linalg.multi_dot([helper.t().float(), between.float(), helper.float()]).item()
    #     last_factor = (1-grid_points[j]**2)**(-(q_hat+1)/2)
    #     p_dec = Decimal(str(p_parameters.p_c_lambda[j].item()))
    #     exp_dec = Decimal(str(eul))**Decimal(str(exponent))
    #     last_dec = Decimal(str(last_factor.item()))
    #     prop = p_dec*exp_dec*last_dec
    #     ps.append(prop)
    #     exps.append(exp_dec)
    # float_tens = torch.tensor(ps, dtype=torch.float)
    # print(float_tens)
    # index = torch.multinomial(torch.abs(float_tens), 1).item()
    # return grid_points[index]
    # return ps
    ps = []
    for j in range(25):
        exponential = math.exp(-0.5*lambda_param*(1-grid_points[j]**2))
        last_factor = (1-grid_points[j]**2)**((q_hat+1)/2)
        ps.append(p_parameters.p_c_lambda[j]*exponential*last_factor)
    tens = torch.tensor(ps, device=device)
    index = torch.multinomial(torch.abs(tens), 1).item()
    return grid_points[index].item()

In [29]:
def step6(omega_squared, kappas, lambdas, U, p_parameters, mu_c, C, q_hat, n, device):
    grid_points = torch.linspace(0, 0.95, 25)
    new_lambda_c_i = []
    for i in range(n):
        new_lambda_c_i.append(iterate_step6(omega_squared, p_parameters=p_parameters, kappa_param=kappas.kappa_c_i[i], lambda_param=lambdas.lambda_c_i[i], S_U=U.S_U_c[i], mu_c=mu_c, C_vec=C[i], q_hat=q_hat, grid_points=grid_points, device=device))
    return new_lambda_c_i

In [30]:
lambdas.lambda_c_i = step6(omega_squared, kappas, lambdas, U, p_parameters, mu_c, C, q_hat, n, device)

## Step 7

In [31]:
def iterate_step7(omega_squared, p_parameters, kappa_param, lambda_param, S_U, G_vec, H_vec, q_hat, grid_points, device):
    # getcontext().prec = 50
    # eul = math.exp(1)
    # ps = []
    # exps = []
    # sum = Decimal('0')
    # for j in range(25):
    #     helper = G_vec-grid_points[j]*H_vec
    #     between = torch.inverse(omega_squared*kappa_param**2*(1-grid_points[j]**2)*S_U)
    #     exponent = -0.5*torch.linalg.multi_dot([helper.t().float(), between.float(), helper.float()]).item()
    #     last_factor = (1-grid_points[j]**2)**(-(q_hat+1)/2)
    #     p_dec = Decimal(str(p_parameters.p_g_lambda[j].item()))
    #     exp_dec = Decimal(str(eul))**Decimal(str(exponent))
    #     last_dec = Decimal(str(last_factor.item()))
    #     prop = p_dec*exp_dec*last_dec
    #     ps.append(prop)
    #     exps.append(exp_dec)
    # float_tens = torch.tensor(ps, dtype=torch.float, device=device)
    # index = torch.multinomial(float_tens, 1).item()
    # return grid_points[index].item()

    ps = []
    for j in range(25):
        exponential = math.exp(-0.5*lambda_param*(1-grid_points[j]**2))
        last_factor = (1-grid_points[j]**2)**((q_hat+1)/2)
        ps.append(p_parameters.p_g_lambda[j]*exponential*last_factor)
    tens = torch.tensor(ps, device=device)
    index = torch.multinomial(torch.abs(tens), 1).item()
    return grid_points[index].item()

In [32]:
def step7(omega_squared, p_parameters, kappas, lambdas, U, G, m, K, q_hat, device):
    grid_points = torch.linspace(0, 0.95, 25)
    new_lambda_g_j = []
    for i in range(m):
        new_lambda_g_j.append(iterate_step7(omega_squared, p_parameters, kappas.kappa_g_j[i], lambdas.lambda_g_j[i], U.S_U_g[i], G[i], U.H[K[i]], q_hat, grid_points, device=device))
    return new_lambda_g_j

In [33]:
lambdas.lambda_g_j = step7(omega_squared, p_parameters, kappas, lambdas, U, G, m, K, q_hat, device)

## Step 8